# mission_to_mars
----

Written in the Python 3.7.9 Environment

By Nicole Lund 

This Jupyter Notebook scrapes Mars space data from various locations for storage in a Pymongo DB and display on a webpage.

In [1]:
# Import Dependencies
import pandas as pd 
import time
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\nlund\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


# NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text.

In [4]:
# Access NASA news site
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)
nasa_html = browser.html
nasa_soup = BeautifulSoup(nasa_html, 'html.parser')

# Allow webpage to load fully
time.sleep(1.5)

In [5]:
# Collect the latest news headline and paragraph
latest_container = nasa_soup.find('div', class_='image_and_description_container')

# Results
news_headline = latest_container.find('div', class_='content_title').find('a').text

news_teaser = latest_container.find('div', class_='article_teaser_body').text

In [6]:
print(news_headline)
print('-------------------')
print(news_teaser)

NASA's Perseverance Rover Begins Its First Science Campaign on Mars
-------------------
The six-wheeled scientist is heading south to explore Jezero Crater’s lakebed in search of signs of ancient microbial life.


# JPL Mars Space Images - Featured Image

* Scrape the [JPL Featured Space Image Website](https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html) for the complete url string of the full size .jpg Featured Mars Image.

In [7]:
# Access JPL image site
jpl_base_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
jpl_url = jpl_base_url + 'index.html'
browser.visit(jpl_url)
jpl_html = browser.html
jpl_soup = BeautifulSoup(jpl_html, 'html.parser')

# Allow webpage to load fully - Has not been necessary
# time.sleep(1)

In [8]:
# Collect the full url path for the full size featured image
featured = jpl_soup.find('div', class_='floating_text_area')
jpl_relative_url = featured.find('a')['href']

# Result
featured_image_url = jpl_base_url + jpl_relative_url

In [9]:
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg


# Mars Facts

* Use Pandas to scrape the table containing facts about the planet from [Mars Facts webpage](https://space-facts.com/mars/) then convert the data to a HTML table string.

In [10]:
# Collect Mars Facts Table
facts_url = 'https://space-facts.com/mars/'
facts_df = pd.read_html(facts_url)[0]

# Convert facts table to html string
facts_html = facts_df.to_html(justify='left',border=0,header=False,index=False)
facts_html

'<table border="0" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [11]:
# Clean up table for Jupyter Notebook Display
facts_df = facts_df.rename(columns={0:'Description',1:'Value'})
facts_df = facts_df.set_index('Description')
facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


# Mars Hemispheres

* Obtain full resolution images and image names for each of Mar's hemispheres from [USGS Astrogeology site](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to a dictionary then store both dictionaries in a list.

In [12]:
# Access Astrogeology site
astropedia_base_url = 'https://astrogeology.usgs.gov'
astropedia_relative_url = '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
astropedia_url = astropedia_base_url + astropedia_relative_url
browser.visit(astropedia_url)
astropedia_html = browser.html
astropedia_soup = BeautifulSoup(astropedia_html, 'html.parser')

# Allow webpage to load fully - Has not been necessary
# time.sleep(1)

In [13]:
# Initialize image list
hemisphere_titles = []

# Find the hemisphere titles containers
hemisphere_containers = astropedia_soup.find_all('div', class_='description')

# Collect hemisphere titles
for image_num in range(0,5):
    try:
        hemisphere_found = hemisphere_containers[image_num].h3.text
        hemisphere_titles.append(hemisphere_found)
        print(f'Found Hemisphere: {hemisphere_found}')
    except:
        print('All Hemispheres Found')

# print('----------------')
# print(hemisphere_titles)

Found Hemisphere: Cerberus Hemisphere Enhanced
Found Hemisphere: Schiaparelli Hemisphere Enhanced
Found Hemisphere: Syrtis Major Hemisphere Enhanced
Found Hemisphere: Valles Marineris Hemisphere Enhanced
All Hemispheres Found


In [14]:
# Navigate to each hemisphere link and collect image link and title in a dictionary
hemisphere_image_urls = []

for hemisphere in hemisphere_titles:
    # Navigate to each hemisphere link
    browser.links.find_by_partial_text(hemisphere).click()

    # Collect URL for full size image
    hemisphere_html = browser.html
    hemisphere_soup = BeautifulSoup(hemisphere_html, 'html.parser')

    # Allow webpage to load fully - Has not been necessary
    # time.sleep(1)

    hemisphere_image = hemisphere_soup.find('img', class_='wide-image')['src']
    image_link = astropedia_base_url + hemisphere_image
    hemisphere_image_urls.append(\
                    {"title":hemisphere,"img_url":image_link})

    # Return to main page
    browser.visit(astropedia_url)

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [15]:
browser.quit()